# Notebook 6.1: ChatGLM2

## 6.1.1 Overview
This is an example shows how to run [ChatGLM2-6B](https://github.com/THUDM/ChatGLM2-6B) Chinese inference on low-cost PCs (without the need of discrete GPU) using [BigDL-LLM](https://github.com/intel-analytics/BigDL/tree/main/python/llm) APIs. ChatGLM2-6B is the second-generation version of the open-source bilingual (Chinese-English) chat model [ChatGLM-6B](https://github.com/THUDM/ChatGLM-6B) proposed by [THUDM](https://github.com/THUDM). ChatGLM2-6B also can be found in [Huggingface models](https://huggingface.co/models) in following [link](https://huggingface.co/THUDM/chatglm2-6b).

Before conducting inference, you may need to prepare environment according to [Chapter 2 Environment Setup](../ch_2_Environment_Setup/README.md).

## 6.1.2 Installation

Install BigDL-LLM through:

In [ ]:
!pip install bigdl-llm[all]

The all option is for installing other required packages by BigDL-LLM.

## 6.1.3 Load Model and Tokenizer

### 6.1.3.1 Load Model

Load model with low-precision optimization(INT4) for lower resource cost using BigDL-LLM APIs, which convert the relevant layers in the model into INT4 format. You can specify the argument `model_path` with both Huggingface repo id or local model path.

> **Note**
>
> BigDL-LLM has supported `AutoModel`, `AutoModelForCausalLM`, `AutoModelForSpeechSeq2Seq` and `AutoModelForSeq2SeqLM`. The AutoClasses help users automatically retrieve the relevant model, in this case, we can simply use `AutoModel` to load.

In [ ]:
from bigdl.llm.transformers import AutoModel

model_path = "THUDM/chatglm2-6b" # repo id or model path
model = AutoModel.from_pretrained(model_path,
                                  load_in_4bit=True,
                                  trust_remote_code=True)

### 6.1.3.2 Load Tokenizer

The quantized model is compatible with the tokenizer provided by the [Huggingface transformers library](https://huggingface.co/docs/transformers/index). Thus, you can directly use Huggingface transformers APIs to load tokenizer. A tokenizer maps between texts and lists of integers. We use it to encode input texts to integers for model to calculate, and decode the model output integers to texts for human to read.

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_path,
                                          trust_remote_code=True)

## 6.1.4 Inference

### 6.1.4.1 Create Prompt Template

Before inference, you need to create a prompt template. Here we give an example prompt template refers to [ChatGLM2-6B prompt template](https://huggingface.co/THUDM/chatglm2-6b/blob/main/modeling_chatglm.py#L1007). You can tune the prompt based on your own model as well.

In [6]:
CHATGLM_V2_PROMPT_TEMPLATE = "问：{prompt}\n\n答："

### 6.1.4.2 Generate

Then, you can generate output with loaded model and tokenizer.

> **Note**
> 
> `max_new_tokens` parameter in the `generate` function defines the maximum number of tokens to predict. 

In [7]:
import time
import torch

prompt = "AI是什么？"
n_predict = 128

with torch.inference_mode():
    prompt = CHATGLM_V2_PROMPT_TEMPLATE.format(prompt=prompt)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    st = time.time()
    output = model.generate(input_ids,
                            max_new_tokens=n_predict)
    end = time.time()
    output_str = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f'Inference time: {end-st} s')
    print('-'*20, 'Output', '-'*20)
    print(output_str)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Inference time: xxxx s
-------------------- Output --------------------
问：AI是什么？

答： AI指的是人工智能,是一种能够通过学习和推理来执行任务的计算机程序。它可以模仿人类的思维方式,做出类似人类的决策,并且具有自主学习、自我进化的能力。AI在许多领域都具有广泛的应用,例如自然语言处理、计算机视觉、机器学习、深度学习等。


### 6.1.4.3 Stream Chat

ChatGLM2-6B support streaming output function `stream_chat`, which enable the model to provide a streaming response word by word. However, other models may not provide similar APIs, if you want to implement general streaming output function, please refer to [Chapter 4.1](../ch_4_Run_Models/4_1_Run_Transformer_Models.ipynb).

In [8]:
import torch

with torch.inference_mode():
    question = "AI 是什么？"
    response_ = ""
    print('-'*20, 'Stream Chat Output', '-'*20)
    for response, history in model.stream_chat(tokenizer, question, history=[]):
        print(response.replace(response_, ""), end="")
        response_ = response

-------------------- Stream Chat Output --------------------
AI指的是人工智能,agogue是一种能够通过学习和理解数据来执行任务的计算机程序。它可以模仿人类的思维方式、学习、推理、理解语言,甚至可以进行一些需要人类直觉和判断的任务。

AI 技术包括机器学习、深度学习、自然语言处理、计算机视觉等,可以应用于各种领域,如医疗、金融、零售、能源、交通等。

AI绿豆是一种具有自主学习、自我进化的能力的人工智能系统,能够通过大量的数据和算法训练,逐渐提高自身的准确度和智能化程度,并且可以模拟出人类类似思维模式来处理各种问题。猖獗AI是一种能够模拟出人类智能的计算机程序,可以进行学习、思考、推理、决策等复杂的任务,并且可以应用于各种领域,如游戏、金融、医疗等。

## 6.1.5 Use in LangChain

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is a widely used framework for developing applications powered by language models. In this section, we will show how to integrate BigDL-LLM with LangChain. You can follow this [instruction](https://python.langchain.com/docs/get_started/installation) to prepare environment for LangChain.

If you need, install LangChain through, or you can refer to [Chapter 5 Langchain Integrations](../ch_5_Langchain_Integrations/README.md):

In [ ]:
!pip install langchain==0.0.184
!pip install -U typing_extensions==4.5.0

### 6.1.5.1 Create Prompt Template

Before inference, you need to create a prompt template. Here we give an example prompt template contains two input variables, `history` and `human_input`. You can tune the prompt based on your own model as well.

In [9]:
CHATGLM_V2_LANGCHAIN_PROMPT_TEMPLATE = """{history}\n\n问：{human_input}\n\n答："""

### 6.1.5.2 Prepare Chain

Use [LangChain API](https://api.python.langchain.com/en/latest/api_reference.html) `LLMChain` to construct a chain for inference. Here we use BigDL-LLM APIs to construct a `LLM` object, which will load model with low-precision optimization automatically.

In [ ]:
from langchain import LLMChain, PromptTemplate
from bigdl.llm.langchain.llms import TransformersLLM
from langchain.memory import ConversationBufferWindowMemory

llm_model_path = "THUDM/chatglm2-6b" # the path to the huggingface llm model

prompt = PromptTemplate(input_variables=["history", "human_input"], template=CHATGLM_V2_LANGCHAIN_PROMPT_TEMPLATE)
max_new_tokens = 128

llm = TransformersLLM.from_model_id(
        model_id=llm_model_path,
        model_kwargs={"trust_remote_code": True},
)

# Following code are complete the same as the use-case
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    llm_kwargs={"max_new_tokens":max_new_tokens},
    memory=ConversationBufferWindowMemory(k=2),
)


### 6.1.5.3 Generate

In [15]:
text = "AI 是什么？"
response_text = llm_chain.run(human_input=text,stop="\n\n")
print(response_text)



> Entering new LLMChain chain...
Prompt after formatting:


问：AI 是什么？

答：
AI指的是人工智能,是一种能够通过学习和理解数据,以及应用数学、逻辑、推理等知识,来实现与人类智能相似或超越人类智能的计算机系统。AI可以分为弱人工智能和强人工智能。弱人工智能是指一种只能完成特定任务的AI系统,比如语音识别或图像识别等;而强人工智能则是一种具有与人类智能相同或超越人类智能的AI系统,可以像人类一样思考、学习和理解世界。目前,AI的应用领域已经涵盖了诸如自然语言处理、计算机视觉、机器学习、深度学习、自动驾驶、医疗健康等多个领域。

> Finished chain.


问：AI 是什么？

答： AI指的是人工智能,是一种能够通过学习和理解数据,以及应用数学、逻辑、推理等知识,来实现与人类智能相似或超越人类智能的计算机系统。AI可以分为弱人工智能和强人工智能。弱人工智能是指一种只能完成特定任务的AI系统,比如语音识别或图像识别等;而强人工智能则是一种具有与人类智能相同或超越人类智能的AI系统,可以像人类一样思考、学习和理解世界。目前,AI的应用领域已经涵盖了诸如自然语言处理、计算机视觉、机器学习、深度学习、自动驾驶、医疗健康等多个领域。
